## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius or on a unix system.

In [1]:
import os
from PyStemmusScope import StemmusScope
from PyStemmusScope import save

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE executable file is located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository. 

#### Update/set config files

In [2]:
# user must provide the correct path
path_to_config_file = "./config_file_template.txt"
path_to_exe_file = "./STEMMUS_SCOPE"

If you run the model locally, you need to first intsall MATLAB Runtime, see instructions in the [readme](https://github.com/EcoExtreML/processing#readme). Then set `LD_LIBRARY_PATH`. To do this, uncomment the cell below and run it:

In [3]:
# # Set LD_LIBRARY_PATH
# matlab_path = !whereis MATLAB
# matlab_path = matlab_path.s.split(": ")[1]
# os.environ['LD_LIBRARY_PATH'] = (
#     f"{matlab_path}/MATLAB_Runtime/v910/runtime/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/os/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/extern/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64")
# print(os.environ['LD_LIBRARY_PATH'])

/usr/local/MATLAB/MATLAB_Runtime/v910/runtime/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/os/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/extern/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64


## Run the model for one forcing file

In [9]:
# create an an instance of the model
model = StemmusScope(config_file=path_to_config_file, exe_file=path_to_exe_file)

# inspect model config template
model.config

{'WorkDir': '/home/sarah/temp/ecoextreml/test/',
 'SoilPropertyPath': '/home/sarah/temp/ecoextreml/data/model_parameters/soil_property/',
 'ForcingPath': '/home/sarah/temp/ecoextreml/data/forcing/plumber2_data/',
 'ForcingFileName': 'AR-SLu_2010-2010_FLUXNET2015_Met.nc',
 'directional': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/soil_spectrum/',
 'InitialConditionPath': '/home/sarah/temp/ecoextreml/soil_initialcondition/',
 'input_data': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/input_data.xlsx',
 'NumberOfTim

In [10]:
# setup the model
config_path = model.setup(
    ForcingFileName="ZA-Kru_2000-2002_FLUXNET2015_Met.nc",
    NumberOfTimeSteps="10",
)
print(config_path)

/home/sarah/temp/ecoextreml/test/input/ZA-Kru_2022-08-03-1537/ZA-Kru_2022-08-03-1537_config.txt


In [11]:
# inspect model configs again
model.config

{'WorkDir': '/home/sarah/temp/ecoextreml/test/',
 'SoilPropertyPath': '/home/sarah/temp/ecoextreml/data/model_parameters/soil_property/',
 'ForcingPath': '/home/sarah/temp/ecoextreml/data/forcing/plumber2_data/',
 'ForcingFileName': 'ZA-Kru_2000-2002_FLUXNET2015_Met.nc',
 'directional': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/soil_spectrum/',
 'InitialConditionPath': '/home/sarah/temp/ecoextreml/soil_initialcondition/',
 'input_data': '/home/sarah/temp/ecoextreml/data/model_parameters/vegetation_property/input_data.xlsx',
 'NumberOfTim

In [12]:
# run the model
result = model.run()
result

b'Reading config from /home/sarah/temp/ecoextreml/test/input/ZA-Kru_2022-08-03-1537/ZA-Kru_2022-08-03-1537_config.txt\n\n The calculations start now \r\n The calculations end now \r'

In [14]:
# save output in nercdf format
variables_will_be_in_NetCDF_file = "./STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv"
nc_file_name = save.to_netcdf(model.config, variables_will_be_in_NetCDF_file)
print(nc_file_name)

/home/sarah/temp/ecoextreml/test/output/ZA-Kru_2022-08-03-1537/ZA-Kru_2022-08-03-1537_STEMMUS_SCOPE.nc


## Run the model for several forcing files

In [15]:
forcing_filenames_list = [
    "ZA-Kru_2000-2002_FLUXNET2015_Met.nc",
    "AR-SLu_2010-2010_FLUXNET2015_Met.nc",
]

full_run = False
if full_run:
    forcing_filenames_list = [file.name for file in Path(model.configs["ForcingPath"]).iterdir()]

for nc_file in forcing_filenames_list:
    # setup the model
    config_path = model.setup(
        ForcingFileName=nc_file,
        NumberOfTimeSteps="10",
    )

    # run the model
    result = model.run()
    print(result)
    
    # save results in a netcdf file
    nc_file_name = save.to_netcdf(model.config, variables_will_be_in_NetCDF_file)
    print(nc_file_name)
    

b'Reading config from /home/sarah/temp/ecoextreml/test/input/ZA-Kru_2022-08-03-1541/ZA-Kru_2022-08-03-1541_config.txt\n\n The calculations start now \r\n The calculations end now \r'
/home/sarah/temp/ecoextreml/test/output/ZA-Kru_2022-08-03-1541/ZA-Kru_2022-08-03-1541_STEMMUS_SCOPE.nc
b'Reading config from /home/sarah/temp/ecoextreml/test/input/AR-SLu_2022-08-03-1541/AR-SLu_2022-08-03-1541_config.txt\n\n The calculations start now \r\n The calculations end now \r'
/home/sarah/temp/ecoextreml/test/output/AR-SLu_2022-08-03-1541/AR-SLu_2022-08-03-1541_STEMMUS_SCOPE.nc
